## References

- https://www.kaggle.com/yasufuminakama/nbme-deberta-base-baseline-train
- https://www.kaggle.com/abebe9849/nbmeexp019?select=itpt.py
- https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-itpt
- https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain

In [ ]:
!nvidia-smi

Sun Feb 27 01:45:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Configurations

In [ ]:
EXP_NAME = "nbme-exp007"
ENV = "colab"
DEBUG_MODE = False
SUBMISSION_MODE = False

In [ ]:
class CFG:
    env=ENV
    exp_name=EXP_NAME
    debug=DEBUG_MODE
    submission=SUBMISSION_MODE
    apex=True
    input_dir=None
    output_dir=None
    library="pytorch"  # ["tf", "pytorch"]
    device="GPU"  # ["GPU", "TPU"]
    competition_name="nbme-score-clinical-patient-notes"
    id_col="id"
    target_col="location"
    pretrained_model_name="microsoft/deberta-base"
    tokenizer=None
    max_len=None
    output_dim=1
    dropout=0.2
    num_workers=4
    batch_size=8
    lr=2e-5
    betas=(0.9, 0.98)
    weight_decay=0.1
    num_warmup_steps_rate=0.1
    batch_scheduler=True
    epochs=5
    n_fold=5
    train_fold=[0, 1, 2, 3, 4]
    seed=71
    gradient_accumulation_steps=1
    max_grad_norm=1000
    print_freq=100
    train=True
    inference=True

In [ ]:
class CFG_For_MLM:
    epochs = 15 # adjust
    learning_rate = 5e-05
    train_batch_size = 8
    gradient_accum_steps = 4
    eval_batch_size = 16
    eval_steps = 7670 # adjust
    block_size = 466 # tokenizerのmax_length
    mlm_prob = 0.15
    fp16 = True

In [ ]:
if CFG.debug:
    CFG.epochs = 2
    CFG.train_fold = [0, 1]

if CFG.submission:
    CFG.train = False
    CFG.inference = True

## Directory Settings

In [ ]:
import sys
from pathlib import Path


print(CFG.env)
if CFG.env == "colab":
    # colab環境
    from google.colab import drive
    drive.mount("/content/drive")
    CFG.input_dir = Path("./drive/MyDrive/00.kaggle/input") / CFG.competition_name
    CFG.output_dir = Path("./drive/MyDrive/00.kaggle/output") / CFG.competition_name / CFG.exp_name
    if not CFG.output_dir.exists():
        CFG.output_dir.mkdir()
    # install packages
    !pip install transformers

elif CFG.env == "local":
    # ローカルサーバ
    CFG.input_dir = Path("../input/") / CFG.competition_name
    CFG.output_dir = Path("../output/") / CFG.competition_name / CFG.exp_name
    if not CFG.output_dir.exists():
        CFG.output_dir.mkdir()

elif CFG.env == "kaggle":
    # kaggle環境
    CFG.input_dir = Path("../input/") / CFG.competition_name
    CFG.output_dir = Path("./")

colab
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gc
import os
import ast
import time
import math
import random
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score, mean_squared_error, f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset

from transformers import BartModel,BertModel,BertTokenizer
from transformers import DebertaModel,DebertaTokenizer
from transformers import RobertaModel,RobertaTokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel,AutoConfig
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import (AutoModel, 
                          AutoModelForMaskedLM,
                          AutoTokenizer,
                          AutoConfig,
                          AdamW,
                          LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer,
                          TrainingArguments)

## Utilities

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything()

## Data Loading

In [ ]:
train = pd.read_csv(CFG.input_dir / "train.csv")
features = pd.read_csv(CFG.input_dir / "features.csv")
patient_notes = pd.read_csv(CFG.input_dir / "patient_notes.csv")
test = pd.read_csv(CFG.input_dir / "test.csv")

train.shape, features.shape, patient_notes.shape, test.shape

((14300, 6), (143, 3), (42146, 3), (5, 4))

In [ ]:
if CFG.debug:
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    print(train.shape)

## Preprocessing

In [ ]:
patient_notes = patient_notes.sample(frac=1.0, random_state=71)   # random shuffle
#n_train = int(len(patient_notes) * 0.8)

pretrain_texts = patient_notes["pn_history"].unique()
print(len(pretrain_texts))

train_pretrain_texts = patient_notes[~patient_notes["case_num"].isin([1, 8])]["pn_history"].unique()
valid_pretrain_texts = patient_notes[patient_notes["case_num"].isin([1, 8])]["pn_history"].unique()
print(len(train_pretrain_texts), len(valid_pretrain_texts))

with open(CFG.output_dir / "train_text.txt", "w") as f:
    texts  = "\n".join(train_pretrain_texts.tolist())
    f.write(texts)
with open(CFG.output_dir / "valid_text.txt", "w") as f:
    texts  = "\n".join(valid_pretrain_texts.tolist())
    f.write(texts)

42146
37142 5004


## Setup tokenizer

In [ ]:
if CFG.submission:
    tokenizer = AutoTokenizer.from_pretrained(Path("../input/") / CFG.exp_name / "tokenizer/")
else:
    tokenizer = AutoTokenizer.from_pretrained(CFG.pretrained_model_name)
    tokenizer.save_pretrained(CFG.output_dir / "tokenizer/")

CFG.tokenizer = tokenizer

## Setup Dataset

In [ ]:
train_dataset = LineByLineTextDataset(
    tokenizer=CFG.tokenizer,
    file_path=CFG.output_dir / "train_text.txt",
    block_size=CFG_For_MLM.block_size)

valid_dataset = LineByLineTextDataset(
    tokenizer=CFG.tokenizer,
    file_path=CFG.output_dir / "valid_text.txt",
    block_size=CFG_For_MLM.block_size)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=CFG.tokenizer, 
    mlm=True, 
    mlm_probability=CFG_For_MLM.mlm_prob)

## Model

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(CFG.pretrained_model_name)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForMaskedLM: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'deberta.embeddings.position_embeddings.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['cls.predictions.decoder.weight', 'cl

## Training

In [ ]:
training_args = TrainingArguments(
    output_dir=CFG.output_dir,
    overwrite_output_dir=True,
    num_train_epochs=CFG_For_MLM.epochs,
    per_device_train_batch_size=CFG_For_MLM.train_batch_size,
    per_device_eval_batch_size=CFG_For_MLM.eval_batch_size,
    learning_rate=CFG_For_MLM.learning_rate,
    gradient_accumulation_steps=CFG_For_MLM.gradient_accum_steps,
    fp16=CFG_For_MLM.fp16,
    eval_steps=CFG_For_MLM.eval_steps,
    save_steps=CFG_For_MLM.eval_steps,
    evaluation_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    prediction_loss_only=True,
    report_to="none",
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

Using amp half precision backend


In [ ]:
trainer.train()
trainer.save_model(CFG.output_dir)

***** Running training *****
  Num examples = 245684
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 4
  Total optimization steps = 115155


Step,Training Loss,Validation Loss
7670,1.184700,2.063766
15340,1.030100,1.977161
23010,0.973100,1.921434
30680,0.909800,1.868359
38350,0.885200,1.870675


***** Running Evaluation *****
  Num examples = 32058
  Batch size = 16
Saving model checkpoint to drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-7670
Configuration saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-7670/config.json
Model weights saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-7670/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 32058
  Batch size = 16
Saving model checkpoint to drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-15340
Configuration saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-15340/config.json
Model weights saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-15340/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 32058
  Batch size = 16
Saving model chec

Step,Training Loss,Validation Loss
7670,1.184700,2.063766
15340,1.030100,1.977161
23010,0.973100,1.921434
30680,0.909800,1.868359
38350,0.885200,1.870675
46020,0.830700,1.848307
53690,0.804200,1.802104
61360,0.780700,1.785465
69030,0.766600,1.771162


***** Running Evaluation *****
  Num examples = 32058
  Batch size = 16
Saving model checkpoint to drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-46020
Configuration saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-46020/config.json
Model weights saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-46020/pytorch_model.bin
Deleting older checkpoint [drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-30680] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 32058
  Batch size = 16
Saving model checkpoint to drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-53690
Configuration saved in drive/MyDrive/00.kaggle/output/nbme-score-clinical-patient-notes/nbme-exp007/checkpoint-53690/config.json
Model weights saved in drive/MyDrive/00.kaggle/output/nbme-score-clini